In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd
from fbprophet import Prophet
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")

C:\Users\kseni\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\kseni\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
#DATA PREPARATION FOR UNIVARIATE PROPHET MODEL

#GATHER Beijing DATA FROM ALL AVAILABLE YEARS
Beijing_Data = pd.read_csv('Beijing_data.csv')
Beijing_Data.rename(columns = {Beijing_Data.columns[0]:'ds'}, inplace = True)

#KEEP ONLY THE POLLUTANTS COLUMNS
cols=['pm25', 'pm10', 'no2', 'o3', 'so2', 'ds']

#MAKE VALUES NUMERIC
Beijing_Data_uni = Beijing_Data.loc[:, Beijing_Data.columns.intersection(cols)]
Beijing_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']] = Beijing_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']].apply(pd.to_numeric, errors='coerce')

#FILL NAN VALUES
Beijing_Data_uni = Beijing_Data_uni.iloc[1: , :]

#SOME VALUES ARE EMPTY, BUT NOT RECOGNIZED AS NAN. REPLACE EMPTY VALUES WITH NAN TO FILL THEM LATER
Beijing_Data_uni = Beijing_Data_uni.replace(r'^\s*$', np.nan, regex=True)
Beijing_Data_uni = Beijing_Data_uni.fillna(method='ffill')
Beijing_Data_uni = Beijing_Data_uni.fillna(method='bfill')

#REMOVE DUPLICATE VALUES
Beijing_Data_uni = Beijing_Data_uni[~Beijing_Data_uni.index.duplicated(keep='first')]

#Fb prophet model predicts univariate time series. It requires that the Date index column is named 'ds'
#and the values column is named 'y'
Beijing_Data_uni['ds'] = pd.DatetimeIndex(Beijing_Data_uni['ds'])

In [3]:
Beijing_Data_uni

,ds,no2,o3,pm10,pm25,so2
1,2014-12-31,0.067385,0.255293,0.037298,0.046053,0.081319
2,2015-01-01,0.319407,0.120797,0.059476,0.311404,0.320879
3,2015-01-02,0.405660,0.069738,0.067540,0.309211,0.356044
4,2015-01-03,0.603774,0.029888,0.118952,0.462719,0.868132
5,2015-01-04,0.761456,0.029888,0.146169,0.530702,0.800000
...,...,...,...,...,...,...
1890,2022-03-15,0.129380,0.287671,0.058468,0.243421,0.010989
1891,2022-03-16,0.117251,0.247821,0.070565,0.250000,0.010989
1892,2022-03-17,0.091644,0.247821,0.030242,0.092105,0.021978
1893,2022-03-18,0.146900,0.120797,0.045363,0.254386,0.021978


In [4]:
#Check data for stationarity to apply the ARIMA model.

#The null hypothesis is that the time series is non-stationary.
#Using the Augmented Dickey Fuller test (adfuller()) we can check for stationarity.
#If p-value < significance level(0.05), the we reject the null hypothesis.
#Otherwise, the time series is non-stationary and needs differencing.

print("Check if Beijing_Data is stationary :")
for col in ['no2', 'o3', 'so2', 'pm10', 'pm25']:
    result = adfuller(Beijing_Data[col])
    print('p-value for ' + col + ' : %f' % result[1])

Check if Beijing_Data is stationary :
p-value for no2 : 0.000000
p-value for o3 : 0.000024
p-value for so2 : 0.000008
p-value for pm10 : 0.000000
p-value for pm25 : 0.000007


In [5]:
def fbprophet_predict(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    
    #SPLIT TO TRAIN AND TEST PORTIONS
    train_size = 0.8
    train = time_series[:int(train_size*(len(time_series)))]
    test = time_series[int(train_size*(len(time_series))):]
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Beijing[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Beijing[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Beijing[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [6]:
#FB PROPHET REQUIRES THE DATAFRAME TO HAVE TWO COLUMNS. 'ds' AND 'y', 'ds' IS THE TIME INDEX AND 'y' IS THE
#VALUE OF THE POLLUTANT

#RENAME COLUMNS TO FIT TO THE PROPHET MODEL
Beijing_o3 = Beijing_Data_uni[['ds', 'o3']]
Beijing_o3.rename(columns = {'o3':'y'}, inplace = True)

Beijing_no2 = Beijing_Data_uni[['ds', 'no2']]
Beijing_no2.rename(columns = {'no2':'y'}, inplace = True)

Beijing_so2 = Beijing_Data_uni[['ds', 'so2']]
Beijing_so2.rename(columns = {'so2':'y'}, inplace = True)

Beijing_pm10 = Beijing_Data_uni[['ds', 'pm10']]
Beijing_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Beijing_pm25 = Beijing_Data_uni[['ds', 'pm25']]
Beijing_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Beijing_o3, Beijing_no2, Beijing_so2, Beijing_pm10, Beijing_pm25]

In [7]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-08  0.220683    0.075023    0.374644
1 2021-02-09  0.218958    0.072876    0.368608
2 2021-02-10  0.222296    0.080563    0.380483
3 2021-02-11  0.224980    0.072021    0.364060
4 2021-02-12  0.224436    0.072504    0.363219
          ds         y
0 2021-02-08  0.191781
1 2021-02-09  0.211706
2 2021-02-10  0.231631
3 2021-02-11  0.282690
4 2021-02-12  0.358655
FB Prophet RMSE for Beijing[o3]: 0.10862795063636295

FB Prophet MSE for Beijing[o3]: 0.011800031659456107

FB Prophet MAE for Beijing[o3]: 0.08626221140513567



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-08  0.095939   -0.052381    0.252025
1 2021-02-09  0.098655   -0.049131    0.248393
2 2021-02-10  0.105329   -0.046214    0.261717
3 2021-02-11  0.107670   -0.041194    0.270513
4 2021-02-12  0.106089   -0.040674    0.254159
          ds         y
0 2021-02-08  0.177898
1 2021-02-09  0.252022
2 2021-02-10  0.258760
3 2021-02-11  0.307278
4 2021-02-12  0.177898
FB Prophet RMSE for Beijing[no2]: 0.09534195698221454

FB Prophet MSE for Beijing[no2]: 0.009090088761198448

FB Prophet MAE for Beijing[no2]: 0.0706001032084929



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-08  0.047536   -0.066432    0.165048
1 2021-02-09  0.048713   -0.061636    0.167035
2 2021-02-10  0.044810   -0.068659    0.161756
3 2021-02-11  0.045951   -0.067372    0.161397
4 2021-02-12  0.042153   -0.067549    0.158744
          ds         y
0 2021-02-08  0.021978
1 2021-02-09  0.054945
2 2021-02-10  0.065934
3 2021-02-11  0.087912
4 2021-02-12  0.087912
FB Prophet RMSE for Beijing[so2]: 0.032052861703986475

FB Prophet MSE for Beijing[so2]: 0.0010273859434148827

FB Prophet MAE for Beijing[so2]: 0.027141319645436025



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-08  0.031368   -0.024701    0.086864
1 2021-02-09  0.032965   -0.023710    0.085939
2 2021-02-10  0.035243   -0.020588    0.087939
3 2021-02-11  0.036398   -0.014668    0.093710
4 2021-02-12  0.034411   -0.018866    0.085891
          ds         y
0 2021-02-08  0.038306
1 2021-02-09  0.054435
2 2021-02-10  0.062500
3 2021-02-11  0.081653
4 2021-02-12  0.108871
FB Prophet RMSE for Beijing[pm10]: 0.05553157170800702

FB Prophet MSE for Beijing[pm10]: 0.003083755456361526

FB Prophet MAE for Beijing[pm10]: 0.02183904727100648

          ds      yhat  yhat_lower  yhat_upper
0 2021-02-08  0.150440   -0.008561    0.323846
1 2021-02-09  0.155433   -0.009520    0.324369
2 2021-02-10  0.155119   -0.012737    0.322259
3 2021-02-11  0.159695   -0.003398    0.327636
4 2021-02-12  0.157283   -0.004425    0.327753
          ds         y
0 2021-02-08  0.217105
1 2021-02-09  0.315789
2 2021-02-10  0.331140
3 2021-02-11  0.502193
4 2021-02-12  0.

### Repeat process but this time train model with the same amount of data as Athens, to prove that the model returns better results due to bigger amount of data.

In [8]:
import pandas as pd
Athens_Data = pd.read_csv('Athens_Data.csv', index_col=0)

In [9]:
len(Athens_Data)

839

In [10]:
data_to_drop = len(Beijing_Data) - len(Athens_Data)

In [11]:
Beijing_less_data = Beijing_Data.iloc[data_to_drop: , :]
len(Beijing_less_data)

839

In [12]:
Beijing_less_data

,ds,no2,o3,pm10,pm25,so2,temp,dew,humidity,windspeed,winddir
1056,2019-11-06,0.357143,0.029888,0.052419,0.206140,0.021978,0.527778,0.656637,0.781755,0.071320,0.252203
1057,2019-11-07,0.239892,0.130760,0.050403,0.146930,0.021978,0.502137,0.483186,0.427252,0.180577,0.261356
1058,2019-11-08,0.350404,0.014944,0.057460,0.210526,0.032967,0.463675,0.534513,0.638568,0.125948,0.606780
1059,2019-11-09,0.405660,0.024907,0.060484,0.315789,0.021978,0.487179,0.612389,0.752887,0.019727,0.273559
1060,2019-11-10,0.086253,0.236613,0.075605,0.166667,0.021978,0.564103,0.575221,0.513857,0.508346,0.917288
...,...,...,...,...,...,...,...,...,...,...,...
1890,2022-03-15,0.129380,0.287671,0.058468,0.243421,0.010989,0.510684,0.539823,0.498845,0.235205,0.279661
1891,2022-03-16,0.117251,0.247821,0.070565,0.250000,0.010989,0.529915,0.571681,0.517321,0.235205,0.323051
1892,2022-03-17,0.091644,0.247821,0.030242,0.092105,0.021978,0.350427,0.460177,0.640878,0.125948,0.394915
1893,2022-03-18,0.146900,0.120797,0.045363,0.254386,0.021978,0.303419,0.518584,0.939954,0.071320,0.397966


In [13]:
Beijing_o3 = Beijing_less_data[['ds', 'o3']]
Beijing_o3.rename(columns = {'o3':'y'}, inplace = True)

Beijing_no2 = Beijing_less_data[['ds', 'no2']]
Beijing_no2.rename(columns = {'no2':'y'}, inplace = True)

Beijing_so2 = Beijing_less_data[['ds', 'so2']]
Beijing_so2.rename(columns = {'so2':'y'}, inplace = True)

Beijing_pm10 = Beijing_less_data[['ds', 'pm10']]
Beijing_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Beijing_pm25 = Beijing_less_data[['ds', 'pm25']]
Beijing_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Beijing_o3, Beijing_no2, Beijing_so2, Beijing_pm10, Beijing_pm25]

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-07  0.500966    0.348686    0.646833
1 2021-09-08  0.497374    0.350705    0.657658
2 2021-09-09  0.487372    0.344507    0.637195
3 2021-09-10  0.504933    0.358255    0.657834
4 2021-09-11  0.514119    0.364371    0.665634
           ds         y
0  2021-09-07  0.353674
1  2021-09-08  0.449564
2  2021-09-09  0.338730
3  2021-09-10  0.424658
4  2021-09-11  0.444583
FB Prophet RMSE for Beijing[o3]: 0.4775622711193123

FB Prophet MSE for Beijing[o3]: 0.2280657227966356

FB Prophet MAE for Beijing[o3]: 0.4602124346075845



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-07  0.068583   -0.051374    0.203257
1 2021-09-08  0.072369   -0.050243    0.195170
2 2021-09-09  0.075199   -0.047740    0.194794
3 2021-09-10  0.080959   -0.048029    0.212114
4 2021-09-11  0.070796   -0.055539    0.202452
           ds         y
0  2021-09-07  0.103774
1  2021-09-08  0.117251
2  2021-09-09  0.141509
3  2021-09-10  0.122642
4  2021-09-11  0.122642
FB Prophet RMSE for Beijing[no2]: 0.20754286579735653

FB Prophet MSE for Beijing[no2]: 0.043074041143379545

FB Prophet MAE for Beijing[no2]: 0.17013436399396115



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-07  0.017651   -0.004419    0.041623
1 2021-09-08  0.016279   -0.006808    0.041152
2 2021-09-09  0.018348   -0.005578    0.041638
3 2021-09-10  0.016975   -0.006023    0.040393
4 2021-09-11  0.017094   -0.005997    0.040498
           ds         y
0  2021-09-07  0.010989
1  2021-09-08  0.010989
2  2021-09-09  0.010989
3  2021-09-10  0.010989
4  2021-09-11  0.021978
FB Prophet RMSE for Beijing[so2]: 0.012585603023569822

FB Prophet MSE for Beijing[so2]: 0.00015839740346688987

FB Prophet MAE for Beijing[so2]: 0.00823106024976557



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-07  0.014351   -0.041823    0.071110
1 2021-09-08  0.016936   -0.041870    0.074368
2 2021-09-09  0.013344   -0.042963    0.068440
3 2021-09-10  0.012040   -0.042146    0.069229
4 2021-09-11  0.014612   -0.043317    0.067386
           ds         y
0  2021-09-07  0.019153
1  2021-09-08  0.033266
2  2021-09-09  0.041331
3  2021-09-10  0.044355
4  2021-09-11  0.055444
FB Prophet RMSE for Beijing[pm10]: 0.06033926951397269

FB Prophet MSE for Beijing[pm10]: 0.003640827445479834

FB Prophet MAE for Beijing[pm10]: 0.05223594899664221

          ds      yhat  yhat_lower  yhat_upper
0 2021-09-07  0.102652   -0.039009    0.235323
1 2021-09-08  0.107429   -0.027230    0.245675
2 2021-09-09  0.111117   -0.042330    0.257325
3 2021-09-10  0.115445   -0.036355    0.254254
4 2021-09-11  0.117651   -0.017779    0.265390
           ds         y
0  2021-09-07  0.081140
1  2021-09-08  0.129386
2  2021-09-09  0.184211
3  2021-09-10  0.184211
4  20

## Seasonal split

The seasonal decomposition of the Beijing data we can see that there is a seasonality in the monthly resampled data. Specifically, for each pollutant the seasonal length of the data is:
- NO2: 12 months or 365 days
- SO2: 12 months
- O3: 12 months
- PM25: 12 months
- PM10: 12 months

In [14]:
def fbprophet_predict_seasonal(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    time_series.index = time_series['ds']
    #SPLIT TO TRAIN AND TEST PORTIONS
    train, test = time_series['2019-12-01' : '2021-12-31'], time_series['2022-01-01' : '2022-03-18']
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Beijing[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Beijing[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Beijing[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [15]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict_seasonal(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.138231    0.009544    0.266634
1 2022-01-02  0.143306    0.007926    0.267076
2 2022-01-03  0.137398    0.005656    0.262025
3 2022-01-04  0.137432    0.005231    0.272805
4 2022-01-05  0.134159    0.002846    0.267630
           ds         y
0  2022-01-01  0.231631
1  2022-01-02  0.226650
2  2022-01-03  0.044832
3  2022-01-04  0.049813
4  2022-01-05  0.019925
FB Prophet RMSE for Beijing[o3]: 0.08333918890918611

FB Prophet MSE for Beijing[o3]: 0.00694542040804101

FB Prophet MAE for Beijing[o3]: 0.07068021623955598



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.205701    0.093693    0.319220
1 2022-01-02  0.190055    0.076366    0.317149
2 2022-01-03  0.184079    0.065800    0.295623
3 2022-01-04  0.192213    0.070159    0.316455
4 2022-01-05  0.197591    0.075408    0.317184
           ds         y
0  2022-01-01  0.307278
1  2022-01-02  0.074124
2  2022-01-03  0.295148
3  2022-01-04  0.326146
4  2022-01-05  0.345013
FB Prophet RMSE for Beijing[no2]: 0.10311059750631736

FB Prophet MSE for Beijing[no2]: 0.010631795318109781

FB Prophet MAE for Beijing[no2]: 0.08751454851002105



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.032285    0.012478    0.052475
1 2022-01-02  0.030109    0.008386    0.051112
2 2022-01-03  0.028868    0.008761    0.049773
3 2022-01-04  0.032583    0.012537    0.053211
4 2022-01-05  0.031772    0.011447    0.052629
           ds         y
0  2022-01-01  0.021978
1  2022-01-02  0.010989
2  2022-01-03  0.010989
3  2022-01-04  0.043956
4  2022-01-05  0.021978
FB Prophet RMSE for Beijing[so2]: 0.01681906660104481

FB Prophet MSE for Beijing[so2]: 0.0002828810013303811

FB Prophet MAE for Beijing[so2]: 0.012605502978574629



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.035383   -0.020314    0.087843
1 2022-01-02  0.033756   -0.019927    0.088608
2 2022-01-03  0.040666   -0.015776    0.096767
3 2022-01-04  0.032203   -0.017658    0.085430
4 2022-01-05  0.035632   -0.021972    0.088926
           ds         y
0  2022-01-01  0.048387
1  2022-01-02  0.007056
2  2022-01-03  0.043347
3  2022-01-04  0.048387
4  2022-01-05  0.053427
FB Prophet RMSE for Beijing[pm10]: 0.028926075599720807

FB Prophet MSE for Beijing[pm10]: 0.0008367178496007635

FB Prophet MAE for Beijing[pm10]: 0.02401253425400172

          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.132038   -0.002432    0.267908
1 2022-01-02  0.123509   -0.016667    0.252272
2 2022-01-03  0.110110   -0.035777    0.246742
3 2022-01-04  0.116468   -0.014623    0.252380
4 2022-01-05  0.127595   -0.002614    0.259076
           ds         y
0  2022-01-01  0.166667
1  2022-01-02  0.037281
2  2022-01-03  0.138158
3  2022-01-04  0.210526
4  